In [ ]:
from typing import Annotated

from dotenv import load_dotenv
from IPython.display import Image
from langchain.chat_models import init_chat_model
from langchain_core.embeddings import DeterministicFakeEmbedding
from langchain_core.messages.ai import AIMessage
from langchain_core.vectorstores import InMemoryVectorStore
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

In [2]:
load_dotenv()  # loads ANTHROPIC_API_KEY as an environment variable

True

In [3]:
# example user data
user_profiles = {
    "john": {"name": "John", "nationality": "US", "language": "English"},
    "wei": {"name": "Wei", "nationality": "CN", "language": "Chinese"},
    "tanaka": {"name": "Tanaka", "nationality": "JP", "language": "Japanese"},
}

In [4]:
# example product catalog (in-memory)
products = [
    {
        "id": 1,
        "name": "Wireless Earbuds",
        "category": "Electronics",
        "price": 59.99,
        "image_url": "http://example.com/earbuds.jpg",
        "product_url": "http://example.com/earbuds",
    },
    {
        "id": 2,
        "name": "Fitness Watch",
        "category": "Electronics",
        "price": 129.99,
        "image_url": "http://example.com/watch.jpg",
        "product_url": "http://example.com/watch",
    },
    {
        "id": 3,
        "name": "Coffee Mug",
        "category": "Kitchen",
        "price": 12.50,
        "image_url": "http://example.com/mug.jpg",
        "product_url": "http://example.com/mug",
    },
]

In [35]:
# create embeddings and vector stores for marketed products
descriptions = [
    p["name"] + " " + p["category"] + " " + str(p["price"]) for p in products
]
embeddings = DeterministicFakeEmbedding(size=4096)
vectors = embeddings.embed_documents(descriptions)
store = InMemoryVectorStore(embeddings)

In [36]:
type(vectors)

list

In [39]:
from langchain_ollama import OllamaEmbeddings

In [40]:
embeddings = OllamaEmbeddings(model="llama3")

In [42]:
embeddings

OllamaEmbeddings(model='llama3', base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [41]:
embeddings.embed_documents(descriptions)

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [37]:
def retrieve_product(query: str) -> str:
    """Return top product matching the query from vector store."""
    docs = store.similarity_search(query, k=1)
    if docs:
        prod = docs[0]  # top result
        return f"Found product: {prod['name']} (${prod['price']})."
    return "No matching products found."

In [38]:
retrieve_product("headphones")

'No matching products found.'

In [ ]:
vector_store = InMemoryVectorStore.from_vectors(doc_vectors, documents=products)

['Wireless Earbuds Electronics 59.99',
 'Fitness Watch Electronics 129.99',
 'Coffee Mug Kitchen 12.5']

In [5]:
class State(TypedDict):
    """The schema class that defines the state."""

    messages: Annotated[list, add_messages]

In [6]:
# structure the chatbot as a "state machine" given the above State class
graph_builder = StateGraph(State)

In [ ]:
# instantiate a ChatModel class
llm = init_chat_model("anthropic:claude-3-7-sonnet-latest")

In [ ]:
def chatbot(state: State) -> dict[str, list[AIMessage]]:
    """Return a structures response from the LLM."""
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
try:
    graph_builder.add_node("chatbot", chatbot)
except ValueError as e:
    print(e)

In [ ]:
try:
    graph_builder.add_edge(START, "chatbot")
except ValueError as e:
    print(e)

In [ ]:
# compile the graph
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
for event in graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

In [ ]:
def detect_language(state: dict) -> dict:
    """Return a language preference given the state."""
    user_id = state.get("user_id")
    profile = user_profiles.get(user_id, {})
    language = profile.get("language", "English")  # default to English
    return {"language": language}  # update state with detected language

In [ ]:
try:  # add the detect_language node in the graph
    workflow.add_node("detect_language", detect_language)
except ValueError as e:
    print(e)

In [ ]:
try:  # add the route from the Start to the detect_lanugage node
    workflow.add_edge(START, "detect_language")
except ValueError as e:
    print(e)

In [ ]:
def language_route(state: dict) -> str:
    return (
        "greet_chinese"
        if state.get("language") == "Chinese"
        else "greet_English"
    )

In [ ]:
workflow.add_conditional_edges("detect_language", language_route)

In [ ]:
def greet_english(state: dict):
    name = user_profiles[state["user_id"]]["name"]
    msg = f"Hello {name}! How can I assist you today?"
    # Append to conversation log
    conv = state.get("conversation", [])
    return {"conversation": conv + [{"role": "assistant", "content": msg}]}

In [ ]:
def greet_chinese(state: dict):
    name = user_profiles[state["user_id"]]["name"]
    # Simple Chinese greeting
    msg = f"你好 {name}！请问今天需要什么帮助？"
    conv = state.get("conversation", [])
    return {"conversation": conv + [{"role": "assistant", "content": msg}]}

In [ ]:
workflow.add_node("greet_english", greet_english)
workflow.add_node("greet_chinese", greet_chinese)
workflow.add_edge("greet_english", END)
workflow.add_edge("greet_chinese", END)

In [ ]:
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)